In [1]:
from midi_to_dataframe import NoteMapper, MidiReader, MidiWriter
import IPython
from IPython.display import Image, IFrame
from PIL import Image
import seaborn as sns
import pandas as pd
import numpy as np
import os
import json
import music21
import pickle

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

2023-02-17 01:07:49.089951: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-17 01:07:51.846203: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/maarij/anaconda3/envs/tf/lib/
2023-02-17 01:07:51.846840: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/maarij/anaconda3/envs/tf/lib/
2023-02-17 01:07:51.846865: W tensorflow/compiler/tf2ten

In [2]:
note_mapping_config_path = "./config/map-to-group.json"
note_mapper = NoteMapper(note_mapping_config_path)

In [3]:
reader = MidiReader(note_mapper)

# getting midi files
filepath = "./datasets/dataset_pop/"
MidiDataDF = pd.DataFrame()
count = 0

for filename in os.listdir(filepath):
    if filename.endswith(".midi"):
        
        # create file path
        count += 1
        #print(count, filename, end = " ")
        fullFilePath = filepath+filename

        # read file as dataframe
        tempDF = reader.convert_to_dataframe(fullFilePath)
        #print(tempDF.shape[0])
        MidiDataDF = MidiDataDF.append(tempDF)


In [4]:
#print(MidiDataDF)

In [43]:
NotesDataDF = MidiDataDF[["notes"]]

In [44]:
# Tokenizing the notes
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(NotesDataDF["notes"]))

In [45]:
# Creating input sequences
sequences = []
for note in NotesDataDF['notes']:
    sequence = tokenizer.texts_to_sequences([note])[0]
    for i in range(1, len(sequence)):
        n_gram_sequence = sequence[:i+1]
        sequences.append

In [46]:
# Padding sequences
max_sequence_len = max([len(sequence) for sequence in sequences])
padded_sequences = np.array(pad_sequences(sequences, maxlen=max_sequence_len, padding='pre'))

ValueError: max() arg is an empty sequence

In [ ]:
# Creating predictors and target
predictors, target = padded_sequences[:,:-1], padded_sequences[:,-1]
target = to_categorical(target, num_classes=len(tokenizer.word_index)+1)